# Collaborative filtering on the MovieLense Dataset

## Learning Objectives
1. Know how to explore the data using BigQuery
2. Know how to use the model to make recommendations for a user
3. Know how to use the model to recommend an item to a group of users

###### This notebook is based on part of Chapter 9 of [BigQuery: The Definitive Guide](https://www.oreilly.com/library/view/google-bigquery-the/9781492044451/ "http://shop.oreilly.com/product/0636920207399.do") by Lakshmanan and Tigani.
### MovieLens dataset
To illustrate recommender systems in action, let’s use the MovieLens dataset. This is a dataset of movie reviews released by GroupLens, a research lab in the Department of Computer Science and Engineering at the University of Minnesota, through funding by the US National Science Foundation.

Download the data and load it as a BigQuery table using:

In [1]:
import os
import tensorflow as tf
PROJECT = "qwiklabs-gcp-00-eeb852ce8ccb" # REPLACE WITH YOUR PROJECT ID

# Do not change these
os.environ["PROJECT"] = PROJECT
os.environ["TFVERSION"] = '2.1'

In [2]:
%%bash
rm -r bqml_data
mkdir bqml_data
cd bqml_data
curl -O 'http://files.grouplens.org/datasets/movielens/ml-20m.zip'
unzip ml-20m.zip
yes | bq rm -r $PROJECT:movielens
bq --location=US mk --dataset \
    --description 'Movie Recommendations' \
    $PROJECT:movielens
bq --location=US load --source_format=CSV \
    --autodetect movielens.ratings ml-20m/ratings.csv
bq --location=US load  --source_format=CSV \
    --autodetect movielens.movies_raw ml-20m/movies.csv

Archive:  ml-20m.zip
   creating: ml-20m/
  inflating: ml-20m/genome-scores.csv  
  inflating: ml-20m/genome-tags.csv  
  inflating: ml-20m/links.csv        
  inflating: ml-20m/movies.csv       
  inflating: ml-20m/ratings.csv      
  inflating: ml-20m/README.txt       
  inflating: ml-20m/tags.csv         
BigQuery error in rm operation: Not found: Dataset qwiklabs-
gcp-00-eeb852ce8ccb:movielens
Dataset 'qwiklabs-gcp-00-eeb852ce8ccb:movielens' successfully created.


rm: cannot remove 'bqml_data': No such file or directory
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  189M  100  189M    0     0  42.8M      0  0:00:04  0:00:04 --:--:-- 44.9M
Upload complete.
Waiting on bqjob_r7942daedc273564e_0000017c9f591eba_1 ... (48s) Current status: DONE   
Upload complete.
Waiting on bqjob_r637b1837611da49f_0000017c9f5a0360_1 ... (1s) Current status: DONE   


## Exploring the data
Two tables should now be available in <a href="https://console.cloud.google.com/bigquery">BigQuery</a>.

Collaborative filtering provides a way to generate product recommendations for users, or user targeting for products. The starting point is a table, <b>movielens.ratings</b>, with three columns: a user id, an item id, and the rating that the user gave the product. This table can be sparse -- users don’t have to rate all products. Then, based on just the ratings, the technique finds similar users and similar products and determines the rating that a user would give an unseen product. Then, we can recommend the products with the highest predicted ratings to users, or target products at users with the highest predicted ratings.

In [3]:
%%bigquery --project $PROJECT
SELECT *
FROM movielens.ratings
LIMIT 10

Downloading: 100%|██████████| 10/10 [00:03<00:00,  3.05rows/s]


,userId,movieId,rating,timestamp
0,70119,71135,2.0,1382338268
1,70154,778,2.0,1165456828
2,70159,76077,2.0,1427154830
3,70159,88744,2.0,1427071630
4,70180,2582,2.0,993051346
5,70189,1198,2.0,997687790
6,70189,3167,2.0,999041110
7,70201,2399,2.0,1222457941
8,70201,4477,2.0,1225333079
9,70204,1960,2.0,983427633


A quick exploratory query yields that the dataset consists of over 138 thousand users, nearly 27 thousand movies, and a little more than 20 million ratings, confirming that the data has been loaded successfully.

In [4]:
%%bigquery --project $PROJECT
SELECT 
    COUNT(DISTINCT userId) numUsers,
    COUNT(DISTINCT movieId) numMovies,
    COUNT(*) totalRatings
FROM movielens.ratings

Downloading: 100%|██████████| 1/1 [00:01<00:00,  1.47s/rows]


,numUsers,numMovies,totalRatings
0,138493,26744,20000263


On examining the first few movies using the query following query, we can see that the genres column is a formatted string:

In [5]:
%%bigquery --project $PROJECT
SELECT *
FROM movielens.movies_raw
WHERE movieId < 5

Downloading: 100%|██████████| 4/4 [00:01<00:00,  3.25rows/s]


,movieId,title,genres
0,3,Grumpier Old Men (1995),Comedy|Romance
1,4,Waiting to Exhale (1995),Comedy|Drama|Romance
2,2,Jumanji (1995),Adventure|Children|Fantasy
3,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy


We can parse the genres into an array and rewrite the table as follows:

In [6]:
%%bigquery --project $PROJECT
CREATE OR REPLACE TABLE movielens.movies AS
    SELECT * REPLACE(SPLIT(genres, "|") AS genres)
    FROM movielens.movies_raw

Query complete after 0.00s: 100%|██████████| 3/3 [00:00<00:00, 1311.68query/s]                        


""


In [7]:
%%bigquery --project $PROJECT
SELECT *
FROM movielens.movies
WHERE movieId < 5

Downloading: 100%|██████████| 4/4 [00:01<00:00,  3.21rows/s]


,movieId,title,genres
0,4,Waiting to Exhale (1995),"[Comedy, Drama, Romance]"
1,3,Grumpier Old Men (1995),"[Comedy, Romance]"
2,2,Jumanji (1995),"[Adventure, Children, Fantasy]"
3,1,Toy Story (1995),"[Adventure, Animation, Children, Comedy, Fantasy]"


## Matrix factorization
Matrix factorization is a collaborative filtering technique that relies on factorizing the ratings matrix into two vectors called the user factors and the item factors. The user factors is a low-dimensional representation of a user_id and the item factors similarly represents an item_id.



In [8]:
%%bigquery --project $PROJECT
SELECT *
-- Note: remove cloud-training-demos if you are using your own model: 
FROM ML.TRAINING_INFO(MODEL `cloud-training-demos.movielens.recommender`)

Downloading: 100%|██████████| 10/10 [00:01<00:00,  7.75rows/s]


,training_run,iteration,loss,eval_loss,duration_ms
0,0,9,0.478570,NaN,335953
1,0,8,0.483399,NaN,170570
2,0,7,0.489912,NaN,339007
3,0,6,0.498437,NaN,156236
4,0,5,0.510123,NaN,311858
5,0,4,0.527014,NaN,167889
6,0,3,0.549901,NaN,419744
7,0,2,0.586216,NaN,175490
8,0,1,0.791212,NaN,347699
9,0,0,67.987718,NaN,355934


What did you get? Our model took an hour to train, and the training loss starts out extremely bad and gets driven down to near-zero over next the four iterations:

<table>
  <tr>
    <th>Iteration</th>
    <th>Training Data Loss</th>
    <th>Evaluation Data Loss</th>
    <th>Duration (seconds)</th>
  </tr>
  <tr>
    <td>4</td>
    <td>0.5734</td>
    <td>172.4057</td>
    <td>180.99</td>
  </tr>
  <tr>
    <td>3</td>
    <td>0.5826</td>
    <td>187.2103</td>
    <td>1,040.06</td>
  </tr>
  <tr>
    <td>2</td>
    <td>0.6531</td>
    <td>4,758.2944</td>
    <td>219.46</td>
  </tr>
  <tr>
    <td>1</td>
    <td>1.9776</td>
    <td>6,297.2573</td>
    <td>1,093.76</td>
  </tr>
  <tr>
    <td>0</td>
    <td>63,287,833,220.5795</td>
    <td>168,995,333.0464</td>
    <td>1,091.21</td>
  </tr>
</table>

However, the evaluation data loss is quite high, and much higher than the training data loss. This indicates that overfitting is happening, and so we need to add some regularization. Let’s do that next. Note the added l2_reg=0.2:

In [ ]:
%%bigquery --project $PROJECT
SELECT *
-- Note: remove cloud-training-demos if you are using your own model: 
FROM ML.TRAINING_INFO(MODEL `cloud-training-demos.movielens.recommender_l2`)

Now, we get faster convergence (three iterations instead of five), and a lot less overfitting. Here are our results:

<table>
  <tr>
    <th>Iteration</th>
    <th>Training Data Loss</th>
    <th>Evaluation Data Loss</th>
    <th>Duration (seconds)</th>
  </tr>
  <tr>
    <td>2</td>
    <td>0.6509</td>
    <td>1.4596</td>
    <td>198.17</td>
  </tr>
  <tr>
    <td>1</td>
    <td>1.9829</td>
    <td>33,814.3017</td>
    <td>1,066.06</td>
  </tr>
  <tr>
    <td>0</td>
    <td>481,434,346,060.7928</td>
    <td>2,156,993,687.7928</td>
    <td>1,024.59</td>
  </tr>
</table>

By default, BigQuery sets the number of factors to be the log2 of the number of rows. In our case, since we have 20 million rows in the table, the number of factors would have been chosen to be 24. As with the number of clusters in K-Means clustering, this is a reasonable default but it is often worth experimenting with a number about 50% higher (36) and a number that is about a third lower (16):



In [9]:
%%bigquery --project $PROJECT
SELECT *
-- Note: remove cloud-training-demos if you are using your own model:
FROM ML.TRAINING_INFO(MODEL `cloud-training-demos.movielens.recommender_16`)

Downloading: 100%|██████████| 10/10 [00:01<00:00,  8.11rows/s]


,training_run,iteration,loss,eval_loss,duration_ms
0,0,9,0.478570,NaN,335953
1,0,8,0.483399,NaN,170570
2,0,7,0.489912,NaN,339007
3,0,6,0.498437,NaN,156236
4,0,5,0.510123,NaN,311858
5,0,4,0.527014,NaN,167889
6,0,3,0.549901,NaN,419744
7,0,2,0.586216,NaN,175490
8,0,1,0.791212,NaN,347699
9,0,0,67.987718,NaN,355934


When we did that, we discovered that the evaluation loss was lower (0.97) with num_factors=16 than with num_factors=36 (1.67) or num_factors=24 (1.45). We could continue experimenting, but we are likely to see diminishing returns with further experimentation.

## Making recommendations

With the trained model, we can now provide recommendations. For example, let’s find the best comedy movies to recommend to the user whose userId is 903. In the query below, we are calling ML.PREDICT passing in the trained recommendation model and providing a set of movieId and userId to carry out the predictions on. In this case, it’s just one userId (903), but all movies whose genre includes Comedy.

In [10]:
%%bigquery --project $PROJECT
SELECT * FROM
ML.PREDICT(MODEL `cloud-training-demos.movielens.recommender_16`, (
  SELECT 
    movieId, title, 903 AS userId
  FROM movielens.movies, UNNEST(genres) g
  WHERE g = 'Comedy'
))
ORDER BY predicted_rating DESC
LIMIT 5

Downloading: 100%|██████████| 5/5 [00:01<00:00,  3.92rows/s]


,predicted_rating,movieId,title,userId
0,11.496418,51632,Atlantis: Milo's Return (2003),903
1,11.431917,127390,Family Guy Presents: Blue Harvest (2007),903
2,11.428360,89939,Gigi (1949),903
3,11.372779,83359,"Play House, The (1921)",903
4,9.750968,97662,Girl in Progress (2012),903


## Filtering out already rated movies
Of course, this includes movies the user has already seen and rated in the past. Let’s remove them.

**TODO 1**: Make a prediction for user 903 that does not include already seen movies.

In [11]:
%%bigquery --project $PROJECT
SELECT * FROM
ML.PREDICT(MODEL `cloud-training-demos.movielens.recommender_16`, (
  WITH seen AS (
    SELECT ARRAY_AGG(movieId) AS movies 
    FROM movielens.ratings
    WHERE userId = 903
  )
  SELECT 
    movieId, title, 903 AS userId
  FROM movielens.movies, UNNEST(genres) g, seen
  WHERE  g = 'Comedy' AND movieId NOT IN UNNEST(seen.movies)
))
ORDER BY predicted_rating DESC
LIMIT 5

Downloading: 100%|██████████| 5/5 [00:01<00:00,  3.81rows/s]


,predicted_rating,movieId,title,userId
0,11.496418,51632,Atlantis: Milo's Return (2003),903
1,11.431917,127390,Family Guy Presents: Blue Harvest (2007),903
2,11.428360,89939,Gigi (1949),903
3,11.372779,83359,"Play House, The (1921)",903
4,9.750968,97662,Girl in Progress (2012),903


For this user, this happens to yield the same set of movies -- the top predicted ratings didn’t include any of the movies the user has already seen.

## Customer targeting

In the previous section, we looked at how to identify the top-rated movies for a specific user. Sometimes, we have a product and have to find the customers who are likely to appreciate it. Suppose, for example, we wish to get more reviews for movieId = 96481 (American Mullet) which has only one rating and we wish to send coupons to the 5 users who are likely to rate it the highest. 

**TODO 2**: Find the top five users who will likely enjoy *American Mullet (2001)*

In [13]:
%%bigquery --project $PROJECT
SELECT * FROM
ML.PREDICT(MODEL `cloud-training-demos.movielens.recommender_16`, (
  WITH allUsers AS (
     SELECT DISTINCT userId
     FROM movielens.ratings
  )
  SELECT 
    96481 AS movieId, 
    (SELECT title FROM movielens.movies WHERE movieId=96481) title,
    userId
  FROM
    allUsers
))
ORDER BY predicted_rating DESC
LIMIT 5

Downloading: 100%|██████████| 5/5 [00:01<00:00,  4.07rows/s]


,predicted_rating,movieId,title,userId
0,156.088272,96481,American Mullet (2001),5514
1,5.433050,96481,American Mullet (2001),63187
2,4.838611,96481,American Mullet (2001),111273
3,4.728069,96481,American Mullet (2001),133172
4,4.609554,96481,American Mullet (2001),107235


### Batch predictions for all users and movies
What if we wish to carry out predictions for every user and movie combination? Instead of having to pull distinct users and movies as in the previous query, a convenience function is provided to carry out batch predictions for all movieId and userId encountered during training. A limit is applied here, otherwise, all user-movie predictions will be returned and will crash the notebook.

In [14]:
%%bigquery --project $PROJECT
SELECT *
FROM ML.RECOMMEND(MODEL `cloud-training-demos.movielens.recommender_16`)
LIMIT 10

Downloading: 100%|██████████| 10/10 [00:01<00:00,  7.15rows/s]


,predicted_rating,userId,movieId
0,2.404522,65792,256
1,2.324626,65792,768
2,3.138003,65792,1792
3,3.429773,65792,1280
4,2.106728,65792,66304
5,2.055739,65792,2048
6,1.417704,65792,131072
7,1.832677,65792,512
8,1.870980,65792,1024
9,3.760160,65792,2304


As seen in a section above, it is possible to filter out movies the user has already seen and rated in the past. The reason already seen movies aren’t filtered out by default is that there are situations (think of restaurant recommendations, for example) where it is perfectly expected that we would need to recommend restaurants the user has liked in the past.

Copyright 2019 Google Inc. Licensed under the Apache License, Version 2.0 (the "License"); you may not use this file except in compliance with the License. You may obtain a copy of the License at http://www.apache.org/licenses/LICENSE-2.0 Unless required by applicable law or agreed to in writing, software distributed under the License is distributed on an "AS IS" BASIS, WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied. See the License for the specific language governing permissions and limitations under the License.